# Multi-agent Examples

In this example I show an example of an agent that writes a story using a series of steps.

1. The user provides an idea for their story.
2. The `Outline Bot` generates an outline for the story including section titles and descriptions.
3. The first section content is created by the `Story Bot` from section title/description and the overall story description.
4. The `Summary Bot` creates a summary of the newly generated chapter.
5. The section summary is passed to the `Story Bot` along with section title/description to generate the next section.
6. All sections follow steps 3-5.
...
7. The sections are combined into a full story.

![Story bot design diagram](https://storage.googleapis.com/public_data_09324832787/story_bot_design.svg)

In [1]:
import sys
sys.path.append('../src/')

import simplechatbot
from simplechatbot.openai_chatbot import OpenAIChatBot

First I create a new chatbot from OpenAI using the API stored in the keychain file.

I also create a new function `stream_it` that prints the result from the LLM as it is received and returns the full result of the LLM call.

In [2]:
keychain = simplechatbot.APIKeyChain.from_json_file('../keys.json')
base_chatbot = OpenAIChatBot.new(
    model_name = 'gpt-4o-mini', 
    api_key=keychain['openai'],
)

def stream_it(chatbot: simplechatbot.ChatBot, msg: str | None = None) -> simplechatbot.ChatResult:
    stream = chatbot.stream(msg, add_to_history=False)
    for chunk in stream:
        print(chunk.content, end='', flush=True)
    return stream.collect()

Next we need to direct the LLM to create an outline for the story. I do this by creating a system prompt describing the task of creating an outline and a pydantic class to direct the LLM on how to structure its response. This is important because the output of the outline bot will be used for generating chapters.

In [3]:
import pydantic

from typing import Optional

#from pydantic import BaseModel, Field
import pydantic

class StoryOutline(pydantic.BaseModel):
    """Outline of the story."""

    story_topic: str = pydantic.Field(description="The topic of the story.")

    part1_title: str = pydantic.Field(description="Title of Part 1 of the story.")
    part1_description: str = pydantic.Field(description="Longer description of part 1.")

    part2_title: str = pydantic.Field(description="Title of Part 2 of the story.")
    part2_description: str = pydantic.Field(description="Longer description of part 2.")

    part3_title: str = pydantic.Field(description="Title of Part 3 of the story.")
    part3_description: str = pydantic.Field(description="Longer description of part 3.")

    def outline_str(self) -> str:
        return f'topic: {self.story_topic}\n\n{self.part1_title}: {self.part1_description}\n\n{self.part2_title}: {self.part2_description}\n\n{self.part3_title}: {self.part3_description}'


system_prompt = '''
The user will provide you a description of a story, and you must create a chapter outline with titles and brief descriptions.
Each section should contain a title and a brief description of what happens in that section.
The sections should all be part of a single narrative ark, but each section should have a complete beginning, middle, and end.
'''
outline_bot = base_chatbot.structbot_from_model(
    output_structure=StoryOutline,
    system_prompt=system_prompt,
)

q = f'Write an outline for a story about two friends who met when they were young and then lost touch. They meet again as adults and have to navigate their new relationship.'
outline: StoryOutline = outline_bot._model.with_structured_output(StoryOutline).invoke(q)
print(outline.outline_str())

topic: Reunion of Childhood Friends

A Chance Encounter: The story begins in a small town during the summer of their childhood. We are introduced to two friends, Zoe and Ethan, who share a bond over their love for adventures and imagination. After a summer filled with memorable experiences like climbing trees, building forts, and exploring the nearby woods, they promise to stay best friends forever. However, as the years go by, Zoe's family moves to another city, and they lose touch, each leading divergent lives. This part sets up their childhood memories and the dynamic of their friendship.

Years Apart: In this section, we explore Zoe and Ethan's lives as they grow up without each other. Zoe becomes an aspiring artist in a bustling city, struggling with her identity and artistic ambitions, while Ethan finds his way into a corporate job in their hometown. We see the impact of their separation as both friends face personal challenges, relationships, and the bittersweet nostalgia for th

Next we create a bot that will generate the actual chapter content based on information generated in the outline and a summary of the previous chapter (if one exists).

In [4]:
chapter_bot_system_prompt = f'''
You are designed to write a single chapter of a larger story based on the following information:

+ Overall story topic: The topic of the full story.
+ Chapter title: The title of the section you are writing.
+ Chapter description: A longer description of what happens in the section.
+ (optional) previous chapter summary: A summary of the previous chapter.

Your responses should only include text that is part of the story. Do not include the chapter title 
or any other information that is not part of the story itself.
'''

def chapter_bot_prompt(
    story_topic: str,
    chapter_title: str,
    chapter_description: str,
    previous_chapter_summary: Optional[str] = None,
) -> str:
    return f'''
    General story topic: "{story_topic}"

    Section title: "{chapter_title}"

    Description: "{chapter_description}"

    Previous chapter summary: "{previous_chapter_summary if previous_chapter_summary is not None else 'No previous chapter - this is the first!'}"
    '''

chapter_bot = base_chatbot.empty().clone(
    system_prompt=chapter_bot_system_prompt,
)

Now we actually create the chatper using the prompt.

In [5]:
cb1 = base_chatbot.empty().clone(
    system_prompt=system_prompt,
)
prompt = chapter_bot_prompt(outline.story_topic, outline.part1_title, outline.part1_description)
chapter1 = stream_it(chapter_bot, prompt)

The soft hum of summer crickets filled the air as the sun dipped low on the horizon, painting the sky in shades of orange and pink. Zoe and Ethan lay sprawled on the warm grass of their secret spot—a sun-drenched clearing deep in the woods behind their houses. A few stray sunbeams filtered through the branches overhead, illuminating their faces in a gentle glow as they shared stories of grand adventures and the legends of strange creatures that roamed the forest.

“Maybe the old tree is a doorway to another world,” Zoe suggested, her brown eyes sparkling with imagination as she pointed to the gnarled oak that stood like a sentinel at the edge of the clearing. “We could be the first explorers to find out what’s on the other side!”

Ethan laughed, adjusting his position to look at her. “And discover treasure? Or dinosaurs? Come on, Zoe! We need to build a team!” His excitement was palpable, and as he spoke, he gestured dramatically, portraying the valiant explorer he aspired to be.

As t

Next I create a bot that will summarize a chapter. We will eventually feed this into the creation of chapter 2.

In [6]:
summary_bot_system_prompt = f'''
You need to create a summary of the story chapter provided to you by the user.
The summary should include names of relevant characters and capture the story arc of the chapter.
'''
summary_bot = base_chatbot.empty().clone(
    system_prompt=summary_bot_system_prompt,
)
ch1_summary = stream_it(summary_bot, f'Chapter text:\n\n{chapter1.content}')

In this chapter, the story follows two childhood friends, Zoe and Ethan, who share a carefree summer filled with imaginative adventures in a secret clearing in the woods. The chapter begins with them enjoying the beauty of the summer evening, dreaming of exploring an old tree they believe might be a doorway to another world. Their playful fantasies lead them to create a bond defined by their pact of "best friends forever."

As summer progresses, the joy of their friendship unfolds through various escapades, transforming Zoe's old fort into a pirate ship and pretending to be space adventurers. However, the idyllic summer comes to an abrupt end when Zoe's parents announce her family's move to a different city due to her father's job. Heartbroken, Zoe struggles with the news and fears losing her best friend Ethan. Despite trying to stay connected through promises of letters and calls, their communication dwindles over time as they grow apart.

As years pass, Ethan immerses himself in scho

Now we actually generate the second chapter using the outline and the previous chapter summary.

In [7]:
prompt = chapter_bot_prompt(
    outline.story_topic,
    outline.part2_title,
    outline.part2_description,
    ch1_summary.content,
)
chapter2 = stream_it(chapter_bot, prompt)

The city lights sparkled like distant stars as Zoe stood at her window, paint-splattered fingers brushing against the cool glass. The vibrant chaos of urban life pulsed outside, a stark contrast to the quiet solitude that wrapped around her heart. It had been years since she left her childhood home, years since she’d shared laughter with Ethan beneath the thick canopy of trees they once claimed as their own. Now, she spent her days inhaling the scents of oil paint and turpentine, pouring her tangled emotions onto canvases that whispered stories of longing and nostalgia.

Zoe’s small studio apartment was a swirling mix of colors, much like her thoughts. The vibrant reds and soothing blues portrayed the turbulence within her: the ambition to succeed as an artist, the fear of failure, and the unsettling impression that she had lost a piece of herself the moment she had stepped onto a different path. Each brush stroke was a testament to her struggles, yet even in her success, there lingere

Create a summary for chapter 2 now.

In [8]:
ch2_summary = stream_it(summary_bot, f'Previous chapter text:\n\n{chapter1.content}')

In this chapter, Zoe and Ethan, best friends, share joyful summer days in a secret clearing in the woods, filled with imagination and adventurous dreams. They fantasize about exploring a mysterious old tree, suggesting it could be a gateway to another world. Their bond is strong, solidified by a pledge to be "best friends forever." 

However, the idyllic summer comes to an abrupt end when Zoe's mother announces that the family will be moving to a different city due to her father's job. The news devastates Zoe, who struggles to comprehend life without her best friend. As they prepare for her departure, tears are shed and promises are made to stay in touch, but the harsh reality of growing up soon dulls their connection. 

As time passes, both Zoe and Ethan lead separate lives, with Ethan focusing on school and sports, while Zoe adjusts to her new surroundings. The intensity of their childhood friendship begins to fade as nostalgia takes over. Yet, the story hints at the possibility of r

Use the summary of chapter 2 and outline information to create chapter 3.

In [9]:
prompt = chapter_bot_prompt(
    outline.story_topic,
    outline.part2_title,
    outline.part2_description,
    ch2_summary.content,
)
chapter3 = stream_it(chapter_bot, prompt)

In the heart of Manhattan, the city buzzed with life outside Zoe’s tiny apartment. The cacophony of honking horns, distant sirens, and the chatter of pedestrians filled her days, yet within her, there was an emptiness as vast as the skyline. The walls were adorned with her artwork, sprawling canvases bursting with colors that reflected her emotions: bold strokes of crimson for her frustrations, soft blues for her hopes, and shadowy grays for moments of despair. Each piece told a story, but none shared the half of her internal struggle. 

Zoe glanced at her easel, where a blank canvas awaited her next stroke. Her mind was cluttered with thoughts of what could have been. She often drifted back to the sun-drenched days in the woods with Ethan, their laughter echoing among the trees as they conjured up worlds beyond their own. Even now, years later, she could feel the tug of their friendship, a bittersweet melody playing in the back of her mind. 

“Hey, Zoe! You coming to the gallery openi

And now we can review the full text of the book.

In [10]:
story = f'''

Overview: {outline.story_topic}

== Chapter 1: {outline.part1_title} ==

{chapter1.content}


== Chapter 2: {outline.part2_title} ==

{chapter2.content}


== Chapter 3: {outline.part3_title} ==

{chapter3.content}

'''

import pprint
pprint.pprint(story)

('\n'
 '\n'
 'Overview: Reunion of Childhood Friends\n'
 '\n'
 '== Chapter 1: A Chance Encounter ==\n'
 '\n'
 'The soft hum of summer crickets filled the air as the sun dipped low on the '
 'horizon, painting the sky in shades of orange and pink. Zoe and Ethan lay '
 'sprawled on the warm grass of their secret spot—a sun-drenched clearing deep '
 'in the woods behind their houses. A few stray sunbeams filtered through the '
 'branches overhead, illuminating their faces in a gentle glow as they shared '
 'stories of grand adventures and the legends of strange creatures that roamed '
 'the forest.\n'
 '\n'
 '“Maybe the old tree is a doorway to another world,” Zoe suggested, her brown '
 'eyes sparkling with imagination as she pointed to the gnarled oak that stood '
 'like a sentinel at the edge of the clearing. “We could be the first '
 'explorers to find out what’s on the other side!”\n'
 '\n'
 'Ethan laughed, adjusting his position to look at her. “And discover '
 'treasure? Or dinosau